In [ ]:
import math

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

In [2]:

sleepschedule = [
    {
        "range": [0, 2],
        "hours": [11, 14]
    },
    {
        "range": [4, 12],
        "hours": [12, 16]
    },
    {
        "range": [3, 5],
        "hours": [10, 13]
    },
    {
        "range": [6, 12],
        "hours": [9, 12]
    },
    {
        "range": [13, 17],
        "hours": [8, 10]
    }
]
usageHoursWeekday = {
    0: 5.168,
    0.5: 4.788,
    1.0: 4.294,
    1.5: 4.018,
    2.0: 3.635,
    2.5: 3.523,
    3.0: 3.287,
    3.5: 3.286,
    4.0: 3.137,
    4.5: 3.189,
    5.0: 3.157,
    5.5: 3.281,
    6.0: 3.431,
    6.5: 3.795,
    7.0: 4.113,
    7.5: 4.695,
    8.0: 5.344,
    8.5: 6.155,
    9.0: 6.668,
    9.5: 7.241,
    10.0: 7.101,
    10.5: 7.404,
    11.0: 7.263,
    11.5: 7.369,
    12.0: 7.206,
    12.5: 7.212,
    13.0: 6.992,
    13.5: 7.231,
    14.0: 7.536,
    14.5: 7.649,
    15.0: 7.497,
    15.5: 7.659,
    16.0: 7.805,
    16.5: 7.864,
    17.0: 7.759,
    17.5: 7.679,
    18.0: 7.527,
    18.5: 7.723,
    19.0: 7.756,
    19.5: 7.882,
    20.0: 7.985,
    20.5: 8.111,
    21.0: 8.019,
    21.5: 7.913,
    22.0: 7.564,
    22.5: 7.093,
    23.0: 6.455,
    23.5: 5.844,
}
weekdayAreaUnderCurve = 0
totalMbWeekdayOriginalCurve = 0
for day in usageHoursWeekday:
    totalMbWeekdayOriginalCurve += usageHoursWeekday[day]
weekdayAreaUnderCurve = totalMbWeekdayOriginalCurve * 0.5
print(weekdayAreaUnderCurve)
usageHoursWeekened = {
    0: 5.745,
    0.5: 5.383,
    1.0: 4.838,
    1.5: 4.525,
    2.0: 4.103,
    2.5: 3.902,
    3.0: 3.584,
    3.5: 3.519,
    4.0: 3.338,
    4.5: 3.339,
    5.0: 3.308,
    5.5: 3.319,
    6.0: 3.364,
    6.5: 3.651,
    7.0: 3.698,
    7.5: 4.041,
    8.0: 4.359,
    8.5: 4.817,
    9.0: 5.202,
    9.5: 5.492,
    10.0: 5.667,
    10.5: 5.939,
    11.0: 5.976,
    11.5: 6.048,
    12.0: 6.283,
    12.5: 6.303,
    13.0: 6.404,
    13.5: 6.601,
    14.0: 6.714,
    14.5: 6.868,
    15.0: 6.965,
    15.5: 6.994,
    16.0: 7.207,
    16.5: 7.864,
    17.0: 7.759,
    17.5: 7.679,
    18.0: 
}

SyntaxError: expression expected after dictionary key and ':' (4293681930.py, line 97)